In [62]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-upda

In [64]:
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [63]:
# Loading the preprocessed ufo dataset
file_path = Path("/content/sample_data/ufo_pop_merge.csv")
df_ufo_data = pd.read_csv(file_path)
df_ufo_data.head(5)

,Unnamed: 0,summary,date_time,shape,duration,stats,text,posted,city_latitude,city_longitude,key,date_str,time_str,year,month_x,day_x,City,State,Year,Population
0,0,Fleet of moving stars. ((Starlink satellites?)),2020-04-17T20:00:00,light,1 hour,Occurred : 4/17/2020 20:00 (Entered as : 04/1...,Fleet of moving stars Looked like moving stars...,2020-06-25T00:00:00,33.5178,-112.3667,Litchfield Park2020-04-17T20:00:00,2020-04-17,20:00:00,2020,4,17,Litchfield Park,AZ,2020,6847
1,1,Star-like UFO and strange sound.,2020-05-17T21:00:00,light,3 minutes,Occurred : 5/17/2020 21:00 (Entered as : 05/1...,Star-like UFO and strange sound had just seen ...,2020-06-25T00:00:00,33.5178,-112.3667,Litchfield Park2020-05-17T21:00:00,2020-05-17,21:00:00,2020,5,17,Litchfield Park,AZ,2020,6847
2,2,A cross shaped object illuminated with lights...,2004-07-20T19:45:00,cross,1 minute,Occurred : 7/20/2004 19:45 (Entered as : 07/2...,A cross shaped object illuminated with lights...,2004-09-29T00:00:00,33.5178,-112.3667,Litchfield Park2004-07-20T19:45:00,2004-07-20,19:45:00,2004,7,20,Litchfield Park,AZ,2004,3964
3,3,to dark to see color.,2004-04-20T08:45:00,triangle,8:45-8:47,Occurred : 4/20/2004 08:45 (Entered as : 04/2...,"to dark to see color. I'm 12 years old, i was ...",2004-08-11T00:00:00,33.5178,-112.3667,Litchfield Park2004-04-20T08:45:00,2004-04-20,08:45:00,2004,4,20,Litchfield Park,AZ,2004,3964
4,4,Strange craft with blinking lights in a solid ...,2005-08-19T20:30:00,triangle,3 minutes,Occurred : 8/19/2005 20:30 (Entered as : 08/1...,Strange craft with blinking lights in a solid ...,2005-10-11T00:00:00,33.5178,-112.3667,Litchfield Park2005-08-19T20:30:00,2005-08-19,20:30:00,2005,8,19,Litchfield Park,AZ,2005,4361


In [65]:
 # Identify all the possible values in the city column
df_ufo_data['City'].unique()

array(['Litchfield Park', 'Tucson', 'Phoenix', 'Gilbert', 'Tempe',
       'Glendale', 'Mesa', 'Bullhead City', 'Peoria', 'Chandler',
       'Sonoita', 'Cottonwood', 'Prescott', 'Payson', 'Page', 'Yuma',
       'Sun City', 'Salome', 'Sierra Vista', 'Prescott Valley', 'Sedona',
       'Anthem', 'Lake Havasu City', 'Florence', 'Apache Junction',
       'Queen Creek', 'Tonopah', 'Show Low', 'Scottsdale', 'Marana',
       'Goodyear', 'Kingman', 'Buckeye', 'Arizona City', 'Cave Creek',
       'Fountain Hills', 'Topock', 'Green Valley', 'Eloy', 'Amado',
       'Nogales', 'Chino Valley', 'Surprise', 'Holbrook', 'Camp Verde',
       'Paradise Valley', 'Maricopa', 'Quartzsite', 'Avondale', 'Bagdad',
       'Morenci', 'Flagstaff', 'Safford', 'Douglas', 'Coolidge',
       'Carefree', 'Casa Grande', 'Gila Bend', 'Tombstone',
       'Huachuca City', 'Clarkdale', 'Pima', 'Patagonia', 'Williams',
       'Sahuarita', 'Globe', 'Benson', 'Tolleson', 'Kearny', 'Wickenburg',
       'Snowflake', 'Winslow', 

In [66]:
# Encode string values into numeric values
city_dict = {'UFO-Litchfield': 0, 'UFO-Tucson': 1, 'UFO-Phoenix':2, 'UFO-Gilbert':3, 'UFO-Tempe':4, 'UFO-Glendale':5, 'UFO-Mesa':6, 'UFO-Bullhead City':7, 'UFO-Peoria':8, 'UFO-Chandler':9, 'UFO-Cottowood':10, 'UFO-Prescott':11, 'UFO-Payson':12, 'UFO-Page':13, 'UFO-Yuma':14, 'UFO-Sun City':15, 'UFO-Salome':16, 'UFO-Sierra Vista': 17, 'UFO-Prescott Valley':18, 'UFO-Sedona':19, 'UFO-Anthem':20, 'UFO-Lake Havasu City':21, 'UFO-Florence':22, 'UFO-Apache Junction':23, 'UFO-Queen Creek':24, 'UFO-Tonopah':25, 'UFO-Show Low':26, 'UFO-Scottsdale':27, 'UFO-Marana':28, 'UFO-Goodyear':29, 'UFO-Kingman':30, 'UFO-Buckeye':31, 'UFO-Arizona City':32, 'UFO-Cave Creek':33, 'UFO-Fountain Hills':34, 'UFO-Topock':35, 'UFO-Green Valley':36, 'UFO-Eloy':37, 'UFO-Amado':38, 'UFO-Nogales':39, 'UFO-Chino Valley':40, 'UFO-Surprise':41, 'UFO-Holbrook':42, 'UFO-Camp Verde':43, 'UFO-Paradise Valley':44, 'UFO-Maricopa':45, 'UFO-Quartzite':46, 'UFO-Avondale':47, 'UFO-Bagdad':48, 'UFO-Morenci':49, 'UFO-Flagstaff':50, 'UFO-Safford':51, 'UFO-Douglas':52, 'UFO-Coolidge':53, 'UFO-Carefree':54, 'UFO-Casa Grande':55, 'UFO-Gila Bend':56, 'UFO-Tombstone':57, 'UFO-Huachuca City':58, 'UFO-Clarkdale':59, 'UFO-Pima':60, 'UFO-Patagonia':61, 'UFO-Williams':62, 'UFO-Sahuarita':63, 'UFO-Globe':64, 'UFO-Benson':65, 'UFO-Tolleson':66, 'UFO-Kearny':66, 'UFO-Wickenburg':67, 'UFO-Snowflake':68, 'UFO-Winslow':69, 'UFO-Parker':70, 'UFO-Willcox':71, 'UFO-San Luis':72, 'UFO-Springerville':73, 'UFO-Whiteriver':74, 'UFO-Wellton':75, 'UFO-Eagar':76, 'UFO-Duncan':77, 'UFO-El Mirage':78, 'UFO-Thatcher':79, 'UFO-Pine':80, 'UFO-Concho':81, 'UFO-Somerton':82, 'UFO-Clifton':83, 'UFO-Many Farms':84}
df2_ufo_data = df_ufo_data.replace({'City': city_dict})

In [67]:
# City names have been replaced with numbers
df2_ufo_data['City'].unique()

array(['Litchfield Park', 'Tucson', 'Phoenix', 'Gilbert', 'Tempe',
       'Glendale', 'Mesa', 'Bullhead City', 'Peoria', 'Chandler',
       'Sonoita', 'Cottonwood', 'Prescott', 'Payson', 'Page', 'Yuma',
       'Sun City', 'Salome', 'Sierra Vista', 'Prescott Valley', 'Sedona',
       'Anthem', 'Lake Havasu City', 'Florence', 'Apache Junction',
       'Queen Creek', 'Tonopah', 'Show Low', 'Scottsdale', 'Marana',
       'Goodyear', 'Kingman', 'Buckeye', 'Arizona City', 'Cave Creek',
       'Fountain Hills', 'Topock', 'Green Valley', 'Eloy', 'Amado',
       'Nogales', 'Chino Valley', 'Surprise', 'Holbrook', 'Camp Verde',
       'Paradise Valley', 'Maricopa', 'Quartzsite', 'Avondale', 'Bagdad',
       'Morenci', 'Flagstaff', 'Safford', 'Douglas', 'Coolidge',
       'Carefree', 'Casa Grande', 'Gila Bend', 'Tombstone',
       'Huachuca City', 'Clarkdale', 'Pima', 'Patagonia', 'Williams',
       'Sahuarita', 'Globe', 'Benson', 'Tolleson', 'Kearny', 'Wickenburg',
       'Snowflake', 'Winslow', 

In [68]:
grouped_df_ufo_data = df2_ufo_data.groupby('City').count()
grouped_df_ufo_data.head()

,Unnamed: 0,summary,date_time,shape,duration,stats,text,posted,city_latitude,city_longitude,key,date_str,time_str,year,month_x,day_x,State,Year,Population
City,,,,,,,,,,,,,,,,,,,
Amado,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Anthem,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
Apache Junction,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39,39
Arizona City,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
Avondale,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24


In [69]:
df2_ufo_data.head()

,Unnamed: 0,summary,date_time,shape,duration,stats,text,posted,city_latitude,city_longitude,key,date_str,time_str,year,month_x,day_x,City,State,Year,Population
0,0,Fleet of moving stars. ((Starlink satellites?)),2020-04-17T20:00:00,light,1 hour,Occurred : 4/17/2020 20:00 (Entered as : 04/1...,Fleet of moving stars Looked like moving stars...,2020-06-25T00:00:00,33.5178,-112.3667,Litchfield Park2020-04-17T20:00:00,2020-04-17,20:00:00,2020,4,17,Litchfield Park,AZ,2020,6847
1,1,Star-like UFO and strange sound.,2020-05-17T21:00:00,light,3 minutes,Occurred : 5/17/2020 21:00 (Entered as : 05/1...,Star-like UFO and strange sound had just seen ...,2020-06-25T00:00:00,33.5178,-112.3667,Litchfield Park2020-05-17T21:00:00,2020-05-17,21:00:00,2020,5,17,Litchfield Park,AZ,2020,6847
2,2,A cross shaped object illuminated with lights...,2004-07-20T19:45:00,cross,1 minute,Occurred : 7/20/2004 19:45 (Entered as : 07/2...,A cross shaped object illuminated with lights...,2004-09-29T00:00:00,33.5178,-112.3667,Litchfield Park2004-07-20T19:45:00,2004-07-20,19:45:00,2004,7,20,Litchfield Park,AZ,2004,3964
3,3,to dark to see color.,2004-04-20T08:45:00,triangle,8:45-8:47,Occurred : 4/20/2004 08:45 (Entered as : 04/2...,"to dark to see color. I'm 12 years old, i was ...",2004-08-11T00:00:00,33.5178,-112.3667,Litchfield Park2004-04-20T08:45:00,2004-04-20,08:45:00,2004,4,20,Litchfield Park,AZ,2004,3964
4,4,Strange craft with blinking lights in a solid ...,2005-08-19T20:30:00,triangle,3 minutes,Occurred : 8/19/2005 20:30 (Entered as : 08/1...,Strange craft with blinking lights in a solid ...,2005-10-11T00:00:00,33.5178,-112.3667,Litchfield Park2005-08-19T20:30:00,2005-08-19,20:30:00,2005,8,19,Litchfield Park,AZ,2005,4361


In [70]:
# New dataframe
new_ufo_data_df = df2_ufo_data

In [71]:
# Data Preperation
 # Save the cleaned DataFrame as a new CSV file for further use
file_path = Path("C:\\Users\\angel\\OneDrive\\Desktop\\Class_Folder\\Resources\\new_ufo_data.csv")
new_ufo_data_df.to_csv(file_path, index=False)